In [14]:
import pandas as pd

# Load historical data
data = pd.read_csv("/Users/saurabhchopda/AlgoTrading/Stock-Prediction-Models/dataset/AMZN.csv", parse_dates=["Date"])
data.set_index("Date", inplace=True)

In [16]:
# Strategy parameters
n = 20  # bottom n% of the day's range

# Calculate the daily range and threshold
data['Range'] = data['High'] - data['Low']

In [18]:
data['Threshold'] = data['Low'] + (n / 100) * data['Range']

In [21]:
# Generate buy signal when Close is in the bottom n% of the range
data['Buy_Signal'] = data['Close'] <= data['Threshold']

In [23]:
# Create a DataFrame to store trades and calculate returns
trades = pd.DataFrame(index=data.index, columns=['Position', 'Entry_Price', 'Exit_Price', 'Returns'])

In [24]:
position_open = False
for i in range(len(data) - 1):
    # Check if buy signal is generated
    if data['Buy_Signal'].iloc[i]:
        if not position_open:
            # Enter a position on today's close
            trades['Position'].iloc[i] = 1
            trades['Entry_Price'].iloc[i] = data['Close'].iloc[i]
            position_open = True
        # If holding, skip the exit process until next signal

    # Check for exit at the next day's close unless signal is repeated
    elif position_open:
        trades['Exit_Price'].iloc[i] = data['Close'].iloc[i]
        position_open = False

/var/folders/9r/gpf3r8sd3f3db5vx30c720j80000gn/T/ipykernel_9585/2008577672.py:7: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  trades['Position'].iloc[i] = 1
/var/folders/9r/gpf3r8sd3f3db5vx30c720j80000gn/T/ipykernel_9585/2008577672.py:8: Fu

In [25]:
trades

,Position,Entry_Price,Exit_Price,Returns
Date,,,,
2022-11-02,1,92.120003,NaN,NaN
2022-11-03,NaN,NaN,NaN,NaN
2022-11-04,NaN,NaN,90.980003,NaN
2022-11-07,NaN,NaN,NaN,NaN
2022-11-08,NaN,NaN,NaN,NaN
...,...,...,...,...
2024-10-28,NaN,NaN,NaN,NaN
2024-10-29,NaN,NaN,190.830002,NaN
2024-10-30,1,192.729996,NaN,NaN


In [26]:
# Calculate returns
trades['Returns'] = trades['Exit_Price'] / trades['Entry_Price'] - 1

# Filter completed trades for analysis
completed_trades = trades.dropna(subset=['Exit_Price'])

In [28]:
completed_trades

,Position,Entry_Price,Exit_Price,Returns
Date,,,,
2022-11-04,NaN,NaN,90.980003,NaN
2022-11-10,NaN,NaN,96.629997,NaN
2022-11-29,NaN,NaN,92.419998,NaN
2022-12-07,NaN,NaN,88.459999,NaN
2022-12-20,NaN,NaN,85.190002,NaN
...,...,...,...,...
2024-10-08,NaN,NaN,182.720001,NaN
2024-10-15,NaN,NaN,187.690002,NaN
2024-10-24,NaN,NaN,186.380005,NaN


In [27]:
# Performance metrics
total_return = completed_trades['Returns'].sum()
average_return = completed_trades['Returns'].mean()
win_rate = (completed_trades['Returns'] > 0).mean()
print("Total Return:", total_return)
print("Average Return per Trade:", average_return)
print("Win Rate:", win_rate)

# Plot performance or output further analysis


Total Return: 0
Average Return per Trade: nan
Win Rate: 0.0
